# isolated_atom - Python class

__Python imports__

- [numpy](http://www.numpy.org/)
- [IPython](https://ipython.org)
- [atomman](https://github.com/usnistgov/atomman)
- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from pathlib import Path
import datetime
from math import floor

# http://www.numpy.org/
import numpy as np

# https://ipython.org/
from IPython.display import display, Markdown

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

Notebook last executed on 2022-02-17 using iprPy version 0.11.2


Import additional libraries for plotting. The external libraries used are:

- [bokeh](http://bokeh.pydata.org/)

In [2]:
import bokeh
print('Bokeh version =', bokeh.__version__)
from bokeh.plotting import figure, output_file, show
from bokeh.embed import components
from bokeh.resources import Resources
from bokeh.io import output_notebook
output_notebook()

Bokeh version = 2.4.2


Loading BokehJS ...

## 1. Load calculation and view description

### 1.1. Load the calculation

In [3]:
# Load the calculation being demoed
calculation = iprPy.load_calculation('E_vs_r_scan')

### 1.2. Display calculation description and theory

In [4]:
# Display main docs and theory
display(Markdown(calculation.maindoc))
display(Markdown(calculation.theorydoc))

# E_vs_r_scan calculation style

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

## Introduction

The E_vs_r_scan calculation style calculation creates a plot of the cohesive energy vs interatomic spacing, $r$, for a given atomic system. The system size is uniformly scaled ($b/a$ and $c/a$ ratios held fixed) and the energy is calculated at a number of sizes without relaxing the system. All box sizes corresponding to energy minima are identified.

This calculation was created as a quick method for scanning the phase space of a crystal structure with a given potential in order to identify starting guesses for further structure refinement calculations.

### Version notes

- 2018-07-09: Notebook added.
- 2019-07-30: Description updated and small changes due to iprPy version.
- 2020-05-22: Version 0.10 update - potentials now loaded from database.
- 2020-09-22: Setup and parameter definitions streamlined.

### Additional dependencies

### Disclaimers

- [NIST disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)
- The minima identified by this calculation do not guarantee that the associated crystal structure will be stable as no relaxation is performed by this calculation. Upon relaxation, the atomic positions and box dimensions may transform the system to a different structure.
- It is possible that the calculation may miss an existing minima for a crystal structure if it is outside the range of $r$ values scanned, or has $b/a$, $c/a$ values far from the ideal.


## Method and Theory

An initial system (and corresponding unit cell system) is supplied. The $r/a$ ratio is identified from the unit cell. The system is then uniformly scaled to all $r_i$ values in the range to be explored and the energy for each is evaluated using LAMMPS and "run 0" command, i.e. no relaxations are performed.

In identifying energy minima along the curve, only the explored values are used without interpolation. In this way, the possible energy minima structures are identified for $r_i$ where $E(r_i) < E(r_{i-1})$ and $E(r_i) < E(r_{i+1})$.


## 2. Specify input parameters

The calculation input parameters can be specified directly to the associated calculation's attributes.  Note that these parameters are often part of calculation subsets allowing multiple calculations to share the same input terms.

### 2.1. LAMMPS-related commands

Calculations that use LAMMPS need to know which LAMMPS executable to use.  Additionally, running LAMMPS in parallel requires specifying an MPI command.

- __commands__ is a LammpsCommands subset.
    - __lammps_command__ is the LAMMPS command to use.
    - __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

In [5]:
calculation.commands.lammps_command = 'lmp_serial'
calculation.commands.mpi_command = None

### 2.2. Interatomic potential

LAMMPS calculations also need to know which potential to use.  These are handled by loaded potentials/atomman PotentialLAMMPS objects.

- __potential__ is a LammpsPotential subset.
    - __potential__ is a loaded PotentialLAMMPS or PotentialLAMMPSKIM object.

In [6]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'
potential = am.load_lammps_potential(id=potential_name, getfiles=True)

# So many "potential"s...
calculation.potential.potential = potential

### 2.3. Initial unit cell system

Many crystal and defect properties start with a core unit cell 

- __system__ is an AtommanSystemLoad subset.
    - __load_ucell()__ is a wrapper around atomman.load() for loading configuration files that also sets the associated object attributes.
    - __ucell__ is the loaded unit cell.

In [10]:
# Create ucell by loading prototype record
ucell=am.load('prototype', 'A1--Cu--fcc', symbols='Ni')

In [18]:
m = ucell.model()
m

DataModelDict([('atomic-system',
                DataModelDict([('box',
                                DataModelDict([('avect',
                                                DataModelDict([('value',
                                                                [1.0,
                                                                 0.0,
                                                                 0.0])])),
                                               ('bvect',
                                                DataModelDict([('value',
                                                                [0.0,
                                                                 1.0,
                                                                 0.0])])),
                                               ('cvect',
                                                DataModelDict([('value',
                                                                [0.0,
                                      

In [20]:
m.append(['atomic-system', 'atom-type-symbol'], 'Ag')

TypeError: unhashable type: 'list'

### 2.4. System modifications

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

- __system__ is an atomman.System to perform the scan on (required). 

In [9]:
sizemults = [3, 3, 3]

# Generate system by supersizing ucell
system = ucell.supersize(*sizemults)
print('# of atoms in system =', system.natoms)

# of atoms in system = 108


### 3.5. Calculation-specific parameters

- __rmin__ is the minimum r spacing to use.
- __rmax__ is the minimum r spacing to use.
- __rsteps__ is the number of r spacing steps to evaluate.

In [10]:
rmin = uc.set_in_units(2.0, 'angstrom')
rmax = uc.set_in_units(6.0, 'angstrom')
rsteps = 200

## 4. Run calculation and view results

### 4.1. Run calculation
 
__runcalc()__ runs the calc method and sets the calculation's inputs and results methods as class attributes.  It takes the calc method's inputs as well as allowing for additional metadata to be specified.  One key difference is that the positional arguments of calc must be explicitly named.

In [11]:
calculation.runcalc(lammps_command = lammps_command,
                    system = system,
                    potential = potential,
                    mpi_command = mpi_command,
                    ucell = ucell, 
                    rmin = rmin, 
                    rmax = rmax, 
                    rsteps = rsteps)

ValueError: load_file not set

### 4.2. Object attributes

#### 4.2.1. Input attributes

In [ ]:
# LammpsCommands subset
print('calculation.commands.lammps_command ->',calculation.commands.lammps_command)
print('calculation.commands.mpi_command ->', calculation.commands.mpi_command)

# LammpsPotential subset
print('calculation.potential.potential_LAMMPS_id ->', calculation.potential.potential_LAMMPS_id)

# Calculation-specific inputs

#### 4.2.2. Results attributes

- __isolated_atom_energy__ (*dict*) contains the computed isolated atom energies for each of the potential's symbol models.  

In [ ]:
energy_unit = 'eV'
for symbol, energy in calculation.isolated_atom_energy.items():
    print(symbol, uc.get_in_units(energy, energy_unit), energy_unit)

### 4.3. Output as JSON/XML

Outputting as JSON or XML requires that the inputs and results be interpreted by the object.

- __build_model()__ will interpret the object's attributes into a data model.
- __model__ is the generated data model.
- __model.json()__ will convert the model to JSON.
- __model.xml()__ will convert the model to XML.

In [ ]:
calculation.build_model()
print(calculation.model.json(indent=4))